In [14]:
import pandas as pd
from openai import OpenAI
import json


In [15]:
#ujraépítjuk az adatokat, innen bármikor el kell indulni
city="budaors"
csv_path='/content/drive/MyDrive/Zenga/'
df_city=pd.read_csv(csv_path+f'zenga_data_{city}.csv')

In [16]:
from google.colab import userdata
key=userdata.get('OpenAi_API')

In [17]:
#Demo szöveg
szoveg='''
Budafok szép panorámás részén a Rózsavölgy feletti dombon, csendes mellékutcában eladó egy bájos kis ház 520 négyzetméteres telken.

A kis ház 35 négyzetméteres, egy légtérben van kialakítva benne a nappali, melyet részben nyitott válaszfallal szeparáltak el a konyha-étkező résztől. A zuhanyzós fürdő és a WC egy helyiségben található.

A víz, az áram és az internet be van kötve a házba. Gáz és csatorna az utcában. A csatorna bevezetését már kezdeményezte a szomszédos telek tulajdonosa.

A telken áll egy másik kis házikó is, ami jelenleg egy 9 négyzetméteres szobaként funkcionál.

A kert szép és gondozott, sok gyümölcsfával. Nagyobb mérete miatt sok lehetőséget rejt. Kialakítható rajta több nagy magaságyás, építhető kerti medence vagy finn szauna is.

További információkért és időpont egyeztetésért kérem hívjon telefononn.
Hitellel vásárlás esetén bankfüggetlen és DÍJMENTES hiteltanácsadó közreműködését biztosítjuk, aki a teljes folyamaton végigkíséri Önt.
Az adásvétel biztonságos és gördülékeny bonyolítása érdekében tapasztalt ingatlan szakjogászaink szolgáltatását is ajánlom.
'''

In [18]:
szoveg=df_city.desc[0]


print(szoveg)
print(df_city.url[0])

Budaörs központjában önálló családi ház eladó.

Budaörsön a Kálvária domb közelében kínálunk eladásra egy 201 nm-es önálló telken álló, egyszintes családi házat.
A telken 2 épület van. A fő épület 50 nm-es, 2 szoba, konyha, fürdőszoba.
A kisebb épület kb. 20 nm-es,1 szoba, konyha, fürdőszoba, teljesen külön tömbben található a telken, melynek az alsó szintjén kb 20 nm-es garázs-tároló található. A telken 2 szint lett kialakítva. A felső kertrész alatt helyezkedik el a pince.
Nagyon kellemes környék a város szívében.
https://www.zenga.hu//ingatlan/elado-csaladi-haz-budaors-ofalu/8299009?page=1&pos=1&cr=50


In [84]:
def create_prompt(szoveg,command="None"):
    prompt_message=[
            {"role": "system",
             "content": f'''
                    Te egy ingatlanhirdetés-elemző nyelvész vagy, komoly ingatlanhirdetési tapasztalattal.
                    Célod, hogy az ingatlan hirdetések tökéletesek legyenek ezért kidolgoztál egy hirdetés értékelési szempontrendszert.
                    Ezek a szempontok alapján tökéletes értékelést tudsz adni az adott hirdetés szövege alapján.
                    A szempontok:
                        1. Érthetőség
                        2. Részletesség / információtartalom
                        3. Szerkezet, logikai felépítés
                        4. Célcsoport megszólítása
                        5. Stílus és nyelvhelyesség
                        6. Előnyök kiemelése
                        7. Negatívumok őszinte kezelése
                        8. Eladásra ösztönzés'''},
            {"role": "user",
             "content": """
                            Értékeld az alábbi ingatlanhirdetés szöveget és az értékelést a scoring mezőbe helyezzed el.
                            Az alábbi szempontok alapján 1–5-ig pontozzad, és hozz létre egy összesített eredményt  1-8 közötti szempontok értékelésének átlagaként ez legyen a 9. Összesítés :
                            Majd a tudásod alapján adj egy alternatív javaslatot az eredeti szöveg javítására, hogy a lehető legjobban megfeleljen a szempontoknak.
                            Az alternatív javaslatot strukturáld, és tördeld a jobb érthetőség érdekében, de markdown formátumot ne használj!
                            Fontos, hogy a saját szempontrendszered szerint az alternatív javaslatod értékelése jobb legyen az eredeti értékelésnél!
                            A válaszod csak érvényes JSON formátumban legyen, pontosan az alábbi struktúrában:

                            {"scoring":
                                {

                                    "Érthetőség": <szám>,
                                    "Részletesség": <szám>,
                                    "Szerkezet": <szám>,
                                    "Célcsoport": <szám>,
                                    "Stílus": <szám>,
                                    "Előnyök": <szám>,
                                    "Negatívumok": <szám>,
                                    "Ösztönzés": <szám>,
                                    "Összesítés": <szám.tizedes>"
                                },
                            "proposal":"Ide kerüljön a javított szöveg javaslatod"
                            }


                    A HIRDETÉS SZÖVEGE:
                        """+f" {szoveg}"}
        ]
    return prompt_message


In [73]:
def create_title_prompt(szoveg: str,commands:str)->str:
    return [
        {
            "role": "system",
            "content": (
            "Te egy tömör szövegíró vagy. KIZÁRÓLAG a megadott hirdetésszöveg alapján "
            "készíts pontosan öt ütős címajánlatot.\n\n"
            "Korlátok:\n"
            "- A kimenet KÖTELEZŐEN érvényes JSON objektum legyen pontosan ebben a formában: {\"titles\": [\"...\", \"...\", \"...\", \"...\", \"...\"]}\n"
            "- Ne használj markdown-t, kód fence-et, extra kulcsokat, magyarázatot.\n"
            "- Minden cím legfeljebb 60 karakter legyen, magyar nyelven, emojik és idézőjelek nélkül.\n"
            "- Ne találj ki a hirdetésben nem szereplő tényeket.\n"
            "- Legyenek egymástól különbözőek (más nézőpont/szövegezés), jól olvashatók és kattintásra ösztönzők.\n"
            "- A címek generálásánál vedd figyelembe a következőket:\n"
            f"-{commands}\n"
            ),
        },
        {
            "role": "user",
            "content": f"INGATLAN HIRDETÉS SZÖVEGE:\n{szoveg}"
        }
    ]


In [74]:
def create_title_command(mood:str, lang:str)->str:
    m=f"Használandó STÍLUS: {mood}\n"
    l=f"A cím szöveg nyelvéhez ezt a nyelvet használjad: {lang}!\n"

    return(m+l)

In [78]:
cmd=create_title_command("Kreatív","magyar")

In [67]:
cmd

'Használandó STÍLUS: professzionális\nA cím szöveg nyelvéhez ezt a nyelvet használjad: angol!\n'

In [68]:
prompt_message=create_title_prompt(szoveg,cmd)


In [69]:
prompt_message

[{'role': 'system',
  'content': 'Te egy tömör szövegíró vagy. KIZÁRÓLAG a megadott hirdetésszöveg alapján készíts pontosan öt ütős magyar címajánlatot.\n\nKorlátok:\n- A kimenet KÖTELEZŐEN érvényes JSON objektum legyen pontosan ebben a formában: {"titles": ["...", "...", "...", "...", "..."]}\n- Ne használj markdown-t, kód fence-et, extra kulcsokat, magyarázatot.\n- Minden cím legfeljebb 60 karakter legyen, magyar nyelven, emojik és idézőjelek nélkül.\n- Ne találj ki a hirdetésben nem szereplő tényeket.\n- Legyenek egymástól különbözőek (más nézőpont/szövegezés), jól olvashatók és kattintásra ösztönzők.\n- A címek generálásánál vedd figyelembe a következőket:\n-Használandó STÍLUS: professzionális\nA cím szöveg nyelvéhez ezt a nyelvet használjad: angol!\n\n'},
 {'role': 'user',
  'content': 'INGATLAN HIRDETÉS SZÖVEGE:\nBudaörs központjában önálló családi ház eladó.\n\nBudaörsön a Kálvária domb közelében kínálunk eladásra egy 201 nm-es önálló telken álló, egyszintes családi házat.\nA te

In [79]:
def get_response(szoveg:str,func,command)->dict:
    '''
    A megadott szöveghez promptot készít (`create_prompt`), elküldi az OpenAI Chat
    Completions API-nak (gpt-4o), majd a választ JSON-ként beolvassa és dict-ként visszaadja.

    Paraméterek
    -----------
    szoveg : str
        A bemeneti szöveg, amelyből a `create_prompt` összeállítja a `messages` listát.

    Visszatérés
    -----------
    dict
        A modell által visszaadott JSON objektum, Python dict-be parse-olva.
    '''
    client = OpenAI(api_key=key)
    prompt_message=func(szoveg,command)
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=prompt_message,
        temperature=0.5,
        max_tokens=7000,
        response_format={ "type": "json_object" }
    )

    content = response.choices[0].message.content
    '''
    # a gpt-4 így adja vissza a json választ, kiszedjük a jsont a felesleges keretből

    if content.startswith("```json"):
        content = content.strip("`").lstrip("json").strip()

    #
    '''
    out_dict = json.loads(content)

    return(out_dict)

In [85]:
resp_data=get_response(prompt_message,create_title_prompt,cmd)

In [86]:
print(resp_data)

{'titles': ['Eladó családi ház Budaörs szívében', 'Két épületes ház Budaörs központjában', 'Budaörsön eladó ház garázzsal és pincével', 'Önálló ház a Kálvária domb közelében', 'Kellemes otthon Budaörs központjában']}


In [88]:
resp_data=get_response(prompt_message,create_prompt,"none")
resp_data["scoring"]

{'Érthetőség': 4,
 'Részletesség': 3,
 'Szerkezet': 3,
 'Célcsoport': 3,
 'Stílus': 4,
 'Előnyök': 3,
 'Negatívumok': 3,
 'Ösztönzés': 3,
 'Összesítés': 3.3}

In [89]:
resp_data["proposal"]

'Eladó családi ház Budaörs központjában, a Kálvária domb közelében. Az ingatlan egy 201 nm-es telken található, amelyen két épület helyezkedik el. Az első, fő épület 50 nm-es, két szobával, konyhával és fürdőszobával rendelkezik. A második, kisebb épület körülbelül 20 nm-es, és egy szobát, konyhát, valamint fürdőszobát tartalmaz, teljesen különálló tömbben. Az alsó szintjén egy 20 nm-es garázs-tároló is található. A kert két szintre oszlik, a felső kertrész alatt pince helyezkedik el. Az ingatlan elhelyezkedése ideális, csendes környezetben, mégis közel a város szívéhez. Tökéletes választás családok számára, akik nyugalmat keresnek, de nem szeretnének lemondani a városi élet előnyeiről. Jöjjön el, nézze meg, és szeressen bele új otthonába!'